In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_excel('./weather2.xlsx')
#增加年月维度
data['日期']=pd.to_datetime(data['日期'],errors='coerce')
data['年']=data['日期'].dt.year
data['月']=data['日期'].dt.month
data['日']=data['日期'].dt.day
data = data.loc[:,['年','月','日','最高温','最低温','天气状况','白天风向','最大风力(白天)']]
data.to_csv('weather3.csv',encoding = 'gbk')
data = pd.read_csv('weather3.csv',encoding = 'gbk')
data['天气状况']=data['天气状况'].apply(lambda x:x.split('/')[0])
print(data)

      Unnamed: 0     年   月   日  最高温  最低温 天气状况 白天风向  最大风力(白天)
0              0  2011   1   1    9    1   多云   北风         6
1              1  2011   1   2   11    4   多云   北风         5
2              2  2011   1   3   10    2   多云   北风         4
3              3  2011   1   4   10    5   多云  东北风         4
4              4  2011   1   5    7    3   小雨  东北风         4
...          ...   ...  ..  ..  ...  ...  ...  ...       ...
4012        4012  2021  12  27    7    1   多云   北风         2
4013        4013  2021  12  28    7    4   小雨   北风         2
4014        4014  2021  12  29   15    4   多云  西北风         2
4015        4015  2021  12  30   15    4    晴   北风         2
4016        4016  2021  12  31   12    5    晴   东风         4

[4017 rows x 9 columns]


In [ ]:
class Data:
    def __init__(self) -> None:
        self.N = int(4017)
        self.file_name = 'weather3.csv'  #原始数据的存储位置
        self.Days = []
        self.parameter = ['year', 'month', 'day','tmax', 'tmin', 'weather', 'wind', 'wind_grade']
        self.Weather = {
            '雪': 0,
            '雨夹雪': 1,
            '阴转雪': 2,
            '雪转阴': 3,
            '多云转雪': 4,
            '雪转晴': 5,
            '小雪': 6,
            '大雨': 7,
            '大到暴雨': 8,
            '大雨转雨': 9,
            '中雨到暴雨': 10,
            '小雨到暴风雨': 11,
            '小雨到暴雨': 12,
            '小雨到大雨转雨': 13,
            '中到大雨': 14,
            '中雨转雨': 15,
            '中雨': 16,
            '暴雨': 17,
            '雨': 18,
            '小雨到大雨': 19,
            '小雨到中雨': 20,
            '雷阵雨': 21,
            '阵雨': 22,
            '小到中雨': 23,
            '小雨转雨': 24,
            '多云转小雨': 25,
            '小雨': 26,
            '小雨转阴': 27,
            '阴到小雨': 28,
            '阴转雨': 29,
            '晴转雨': 30,
            '阴转小雨': 31,
            '小雨转多云': 32,
            '多云转雨': 33,
            '晴转小雨': 34,
            '雾': 35,
            '多云': 36,
            '多云转阴': 37,
            '阴': 38,
            '阴转多云': 39,
            '霾': 40,
            '霾转多云': 41,
            '霾转晴': 42,
            '浮尘': 43,
            '多云转晴': 44,
            '晴转多云': 45,
            '阴转晴': 46,
            '晴': 47,
        }
        self.Wind = {
            '西南风': 0,
            '西北风': 1,
            '东北风': 2,
            '西风': 3,
            '北风': 4,
            '东风': 5,
            '南风': 6,
            '东南风': 7,
            '无持续风向':8,
        }

    def _processData(self):
        data = pd.read_csv(self.file_name,encoding = 'gbk')
        data['天气状况']=data['天气状况'].apply(lambda x:x.split('/')[0])
        year=data['年']
        month=data['月']
        day=data['日']
        tmax=data['最高温']
        tmin=data['最低温']
        weather=data['天气状况']
        wind=data['白天风向']
        wind_grade=data['最大风力(白天)']
        for i in range(self.N):
            Day = {
                'year':year[i],
                'month': month[i],
                'day': day[i],
                'tmax': tmax[i],
                'tmin': tmin[i],
                'weather': self.Weather[weather[i]],
                'wind': self.Wind[wind[i]],
                'wind_grade': wind_grade[i],
            }
            self.Days.append(Day)
    def _getData(self):
        with open('train_data.txt', "w") as f:
            for day in self.Days:
                line = []
                for i in self.parameter:
                    line.append(str(day[i]))
                tmax = int(day['tmax'])
                tmin = int(day['tmin'])
                ave_t = str(int((tmax + tmin) / 2))
                line.append(ave_t)
                f.write(' '.join(line) + '\n')
            f.close()

    def read(self):
        self._processData()
        self._getData()

data = Data()
data.read()

In [ ]:
import numpy as np

N, M = 1713, 5
X, Y = [], []
data = []

def getData():
    with open('train_data.txt', 'r') as f:
        for j in range(N):
            day = f.readline()
            day = day.split()
            x = []
            if j >= M:
                Y.append([int(day[-1])])
            for i in range(1, len(day), 1):
                x.append(int(day[i]))
            data.append(x)
        f.close()

getData()


for i in range(N - M):
    x = []
    for j in range(i, i + M):
        for k in range(len(data[i])):
            x.append(data[j][k])
    X.append(x)

Y = np.array(Y)
X = np.array(X)

R = np.dot(X.T, X)
R = np.linalg.inv(R)
X = np.dot(R, X.T)
X = np.dot(X, Y)
print(X)

while True:
    x = []
    for i in range(M):
        line = input()
        line = line.split()
        for j in range(1, len(line)):
            x.append(int(line[j]))
    x = np.array([x])
    print(np.dot(x, X))

[[ 1.66041529e-02]
 [ 5.27983710e-03]
 [ 8.02822135e-02]
 [ 7.97088303e-02]
 [-1.76839746e-03]
 [ 1.25228016e-02]
 [ 1.22970459e-01]
 [-6.21790600e-02]
 [-3.48205194e-02]
 [-4.52668290e-03]
 [-4.84985598e-02]
 [-6.26980091e-03]
 [ 7.62496859e-04]
 [-2.75347258e-04]
 [ 2.70485868e-02]
 [ 1.04336730e-01]
 [ 4.44359726e-03]
 [-1.02992149e-02]
 [-3.97982554e-03]
 [-1.32795837e-01]
 [-4.23038044e-03]
 [-3.17673165e-04]
 [-3.55359826e-02]
 [ 1.26702975e-01]
 [ 2.25921743e-02]
 [ 3.54399586e-03]
 [ 1.51757055e-02]
 [-2.42349718e-01]
 [-2.38961550e-02]
 [-6.80862714e-02]
 [ 1.42110218e-01]
 [ 7.33525871e-02]
 [-4.70605351e-02]
 [ 1.30509072e-02]
 [ 4.19226754e-01]
 [ 9.18335159e-01]
 [ 3.57922253e-02]
 [ 1.62391196e-02]
 [-1.39044369e-01]
 [-3.27384943e-01]]
